# Oliver Medgyesi

## Research question/interests

I am mainly interested in resarching the average salary of an individual, more specifically kelowna residents, and how affordable the current housing market is. I will also determine what factors are most significant in impacting the price of housing and how we can attempt to lower them. Columns 48, 52, and 54 from our dataset will be used to analyze my resarch question.

### My Primary Question:

1. **How does the affordability of housing in Kelowna vary relative to the average monthly net salary, and what factors such as population density, location and demand may be driving or affecting these discrepancies.**

> To answer this question, I am going to analyze the data on the average monthly net saraly and the cost of apartments in various canadian locations. I will then calculate the average cost of housing as a percentage of the varegae salary in each location. I will also focus on the considerable changes in the affordability of housing in Kelowna. I will also explore the potential factors that possibly contribute to these differences.
---
### Supporting Questions Regarding Housing Affordability and the Average Salary:


2. **Is there a correlation between the price of housing and the cost of living and is it impacted by outside factors such as groceries, transportation, and gasoline prices?**
 
 > To answer this question, we will need to find if there is a correlation between the price of housing and the cost of living. We can use regression to examine the relationships between housing prcies and the multiple factors listed. By anaylyzing this way, we will be able to determine if there is any relationship bonded from these categories. If so, we will determine which factor has the greatest impact on the price of housing. 

3. **What percentage of the average salary in Kelowna is required to afford the current housing prices and how has this changed overtime?**

 > I am going to focus on examining the trends over time to determine if there is any correlation between the changes in kelowna residents average salary and the changes in housing affordability. By examining this data over different intervals of time, I will find if there are any significant changes or fluxuations in the housing market.




---


### Data dictionary
| Column | Description |
| - | - |
| city | Name of the city |
| country | Name of the country|
| x1 | Meal, Inexpensive Restaurant (USD) |
| x2 | Meal for 2 People, Mid-range Restaurant, Three-course (USD) |
| x3 | McMeal at Mcdonalds (or Equivalent Combo Meal) (USD) |
| x4 | Domestic Beer (0.5 litre draught, in restaurants) (USD) |
| x5 | Imported Beer (0.33 litre bottle, in restaurants) (USD) |
| x6 | Cappuccino (regular, in restaurants) (USD) |
| x7 | Coke/Pepsi (0.33 litre bottle, in restaurants) (USD) |
| x8 | Water (0.33 litre bottle, in restaurants) (USD) |
| x9 | Milk (regular), (1 litre) (USD) |
| x10 | Loaf of Fresh White Bread (500 g) (USD) |
| x11 | Rice (white), (1kg) (USD) |
| x12 | Eggs (regular) (12) (USD) |
| x13 | Local Cheese (1kg) (USD) |
| x14 | Chicken Fillets (1kg) (USD) |
| x15 | Beef Round (1kg) (or Equivalent Back Leg Red Meat) (USD) |
| x16 | Apples (1kg) (USD) |
| x17 | Bananas (1kg) (USD) |
| x18 | Oranges (1kg) (USD) |
| x19 | Tomatoes (1kg) (USD) |
| x20 | Potatoes (1kg) (USD) |
| x21 | Onions (1kg) (USD) |
| x22 | Lettuce (1 head) (USD) |
| x23 | Water (1.5 litre bottle, at the market) (USD) |
| x24 | Bottle of Wine (Mid-Range, at the market) (USD) |
| x25 | Domestic Beer (0.5 litre bottle, at the market) (USD) |
| x26 | Imported Beer (0.33 litre bottle, at the market) (USD) |
| x27 | Cigarettes 20 Pack (Marlboro) (USD) |
| x28 | One-way Ticket (Local Transport) (USD) |
| x29 | Monthly Pass (Regular Price) (USD) |
| x30 | Taxi Start (Normal Tariff) (USD) |
| x31 | Taxi 1km (Normal Tariff) (USD) |
| x32 | Taxi 1hour Waiting (Normal Tariff) (USD) |
| x33 | Gasoline (1 litre) (USD) |
| x34 | Volkswagen Golf 1.4 90 KW Trendline (Or Equivalent New Car) (USD) |
| x35 | Toyota Corlla Sedan 1.6l 97kW Comfort (Or Equivalent New Car) (USD) |
| x36 | Basic (Electricity, Heating, Cooling, Water, Garbage) for 85 m<sup>2</sup> Apartment (USD) |
| x37 | 1 min. of Prepaid Mobile Tariff Local (No Discounts or Plans) (USD) |
| x38 | Internet (60 Mbps or More, Unlimited Data, Cable/ADSL) (USD) |
| x39 | Fitness Club, Monthly Fee for 1 Adult (USD) |
| x40 | Tennis Court Rent (1 Hour on Weekend) (USD) |
| x41 | Cinema, International Release, 1 Seat (USD) |
| x42 | Preschool (or Kindergarten), Full Day, Private, Monthly for 1 Child (USD) |
| x43 | International Primary School, Yearly for 1 Child (USD) |
| x44 | 1 Pair of Jeans (Levis 501 Or Similar) (USD) |
| x45 | 1 Summer Dress in a Chain Store (Zara, H&M, or similar) (USD) |
| x46 | 1 Pair of Nike Running Shoes (Mid-Range) (USD) |
| x47 | 1 Pair of Men Leather Business Shoes (USD) |
| x48 | Apartment (1 bedroom) in City Centre (USD) |
| x49 | Apartment (1 bedroom) Outside of Centre (USD) |
| x50 | Apartment (3 bedrooms) in City Centre (USD) |
| x51 | Apartment (3 bedrooms) Outside of Centre (USD) |
| x52 | Price per Square Metre to Buy Apartment in City Centre (USD) |
| x53 | Price per Square Metre to Buy Apartment Outside of Centre (USD) |
| x54 | Average Monthly Net Salary (After Tax) (USD) |
| x55 | Mortgage Interest Rate in Percentages (%), Yearly, for 20 Years Fixed-Rate
| data_quality | 0 if Numbeo considers that more contributors are needed to increase data quality, else 1 |

In [ ]:
#imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
#load data
cost_living_df = pd.read_csv("../data/raw/cost-of-living_v2.csv")
cost_living_df.head()

In [ ]:
cost_living_df[cost_living_df['city']=='Kelowna']